In [1]:
import os
import librosa
import librosa.display
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt

In [2]:
def padd_audio(audio, max_duration=5, sample_rate=16000):
    """This function take an audio and padd that audio with zeros"""
    
    max_length = max_duration * sample_rate
    padding_needed = max_length - len(audio)
    pad_left = padding_needed // 2
    pad_right = padding_needed - pad_left
    
    return np.pad(audio, (pad_left, pad_right), 'constant')

In [3]:
def mix_audio(original_audio_path, noise_audio_path, sample_rate=16000):
    """This function take an original audio path and noise audio path and mix it together"""
    
    # Load the original audio
    original_audio, sr = librosa.load(original_audio_path, sr=sample_rate)
    
    #Padd original audio
    original_audio = padd_audio(original_audio, sample_rate=sample_rate)
    
    # Load the noise audio
    noise_audio, sr_noise = librosa.load(noise_audio_path, sr=sample_rate)
    
    # Repeat the noise audio
    noise_audio = np.tile(noise_audio, int(np.ceil(len(original_audio) / len(noise_audio))))

    # Trim the repeated noise audio to match the length of the original audio
    noise_audio = noise_audio[:len(original_audio)]
    
    return original_audio + noise_audio/2

In [4]:
def get_clean_audio(original_audio_path, sample_rate=16000):
    """This function take an original audio path and return padded audio"""
    
    # Load the original audio
    original_audio, sr = librosa.load(original_audio_path, sr=sample_rate)
    
    #Padd original audio
    return padd_audio(original_audio, sample_rate=sample_rate)

In [5]:
sr = 16000  # Sampling rate   

In [6]:
# Randmly chosen noise for each audio
def combine_audio_with_noise(original_audio_dir, noise_audio_dir):
    combination_dict = {}
    noise_audios = os.listdir(noise_audio_dir)
    original_audios = os.listdir(original_audio_dir)
    
    for original_audio in original_audios:
        noise_audio = np.random.choice(noise_audios)
        combination_dict[os.path.join(original_audio_dir, original_audio)] = os.path.join(noise_audio_dir, noise_audio)
        
    return combination_dict

In [7]:
def audio_to_stft(audio, n_fft=1199, hop_length_fft=304):
    # STFT transformation
    stftaudio = librosa.stft(audio, n_fft=n_fft, hop_length=hop_length_fft)

    # Extract magnitude and phase
    stftaudio_magnitude, stftaudio_phase = librosa.magphase(stftaudio)

    # Convert magnitude to dB
    stftaudio_magnitude_db = librosa.amplitude_to_db(stftaudio_magnitude, ref=np.max)
    
    return stftaudio_magnitude_db, stftaudio_phase

In [10]:
def stft_to_audio(stftaudio_magnitude_db, stftaudio_phase, hop_length_fft=304):
# Convert dB back to amplitude
    stftaudio_magnitude_rev = librosa.db_to_amplitude(stftaudio_magnitude_db, ref=1.0)

    # Reconstruct the STFT complex matrix
    audio_reverse_stft = stftaudio_magnitude_rev * stftaudio_phase

    # Inverse STFT to get back the audio signal
    audio_reconstruct = librosa.istft(audio_reverse_stft, hop_length=hop_length_fft)

    return audio_reconstruct / np.max(np.abs(audio_reconstruct))

In [11]:
# Example of normalization function using global min and max values
def normalize(stft, global_min, global_max):
    return (stft - global_min) / (global_max - global_min)

# Example of denormalization function using global min and max values
def denormalize(normalized_stft, global_min, global_max):
    return normalized_stft * (global_max - global_min) + global_min

# Preprocessing the data

In [12]:
audio_noise_pairs = combine_audio_with_noise(os.path.join(os.getcwd(), 'Dataset'), os.path.join(os.getcwd(), 'Noise'))

#Getting all the noisy and clean audio matrices
noisy_audios = np.zeros(len(audio_noise_pairs), dtype=object)
clean_audios = np.zeros(len(audio_noise_pairs), dtype=object)

for index, (audio_dir, noise_dir) in enumerate(audio_noise_pairs.items()):
    noisy_audios[index] = mix_audio(audio_dir, noise_dir, sample_rate=16000)
    clean_audios[index] = get_clean_audio(audio_dir, sample_rate=16000)

In [13]:
#Getting STFT data for learning process
noisy_audios_stft = [audio_to_stft(audio)[0] for audio in noisy_audios]
noisy_audios_stft_phase = [audio_to_stft(audio)[1] for audio in noisy_audios]
clean_audios_stft = [audio_to_stft(audio)[0] for audio in clean_audios]

In [14]:
#Splitting the data for training and test set
split_ratio = 0.9
split_index = int(len(noisy_audios_stft) * split_ratio)

X_train = noisy_audios_stft[:split_index]
X_test = noisy_audios_stft[split_index:]

y_train = clean_audios_stft[:split_index]
y_test = clean_audios_stft[split_index:]

phase_training = noisy_audios_stft_phase[:split_index]
phase_test = noisy_audios_stft_phase[split_index:]

In [15]:
# Find the global min and max values from both the noisy and clean STFT data
global_min = min(np.min(X_train), np.min(y_train))
global_max = max(np.max(X_train), np.max(y_train))

In [16]:
#Normalize data based on the data from training set
X_train_normalized = normalize(X_train, global_min, global_max)
X_test_normalized = normalize(X_test, global_min, global_max)
y_train_normalized = normalize(y_train, global_min, global_max)
y_test_normalized = normalize(y_test, global_min, global_max)

In [17]:
#Transform data to tensors: sample x dimensions(n x n) x channel(1)
X_train_normalized = X_train_normalized[..., np.newaxis]
X_test_normalized = X_test_normalized[..., np.newaxis]
y_train_normalized = y_train_normalized[..., np.newaxis]
y_test_normalized = y_test_normalized[..., np.newaxis]

In [18]:
#np.savez('input_data.npz', array1=X_train_normalized, array2=X_test_normalized, array3=y_train_normalized, array4=y_test_normalized, array5=phase_training, array6=phase_test, array7=global_min, array8=global_max)

# Load the preprocessed data

In [19]:
#data = np.load('input_data.npz')

In [20]:
#X_train_normalized = data['array1']
#X_test_normalized = data['array2']
#y_train_normalized = data['array3']
#y_test_normalized = data['array4']
#phase_training = data['array5']
#phase_test = data['array6']
#global_min = data['array7']
#global_max = data['array8']

In [21]:
def generator(X, y, batch_size=16, epochs=20):
    assert len(X) == len(y), "The length of X and y must be the same"
    
    # Iterate through each epoch
    for epoch in range(epochs):
        indices = np.arange(len(X))
        np.random.shuffle(indices)
        X_shuffled = X[indices]
        y_shuffled = y[indices]
        
        for start in range(0, len(X), batch_size):
            end = min(start + batch_size, len(X))
            batch_X = X_shuffled[start:end]
            batch_y = y_shuffled[start:end]
            
            yield batch_X, batch_y

In [22]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, UpSampling2D, BatchNormalization, MaxPooling2D, Input
from tensorflow.keras.models import Model

# Encoder
def encoder(inputs): #(201, 1201, 1)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2, 2))(x)
    
    encoded = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    return encoded

# Decoder
def decoder(encoded):
    x = UpSampling2D((2, 2))(encoded)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)

    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)  # Single channel output
    return decoded

# Input shape
input_shape = (600, 264, 1)
inputs = Input(shape=input_shape)

# Build the autoencoder
encoded = encoder(inputs)
decoded = decoder(encoded)

autoencoder = Model(inputs, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# Summary of the model
autoencoder.summary()

batch_size = 16
# Fit the model (example)
autoencoder.fit(generator(X_train_normalized, y_train_normalized),
                          steps_per_epoch=len(X_train_normalized) // batch_size,
                          epochs=20)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 600, 264, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 600, 264, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 300, 132, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 300, 132, 64)      18496     
                                                                 
 up_sampling2d (UpSampling2D  (None, 600, 264, 64)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 600, 264, 32)      18464 

In [71]:
def get_audio_predictions(output_prediction, phase, global_min=global_min, global_max=global_max):
    prediction = denormalize(output_prediction.squeeze(), global_min, global_max)
    audio_prediction = stft_to_audio(prediction, phase, hop_length_fft=304)
    return audio_prediction

In [31]:
predictions = autoencoder.predict(X_test_normalized)

3/3 [==============================] - 3s 774ms/step


In [72]:
audio_pred = get_audio_predictions(predictions[2], phase_test[2])

In [73]:
sf.write('audio_pred.wav', audio_pred, samplerate=sr)

In [75]:
sf.write('audio_noisy.wav', stft_to_audio(X_test[2], phase_test[2], hop_length_fft=304), samplerate=sr)